# Word Mover's Distance

In [16]:
import csv
import pickle as pkl

import numpy as np
import pandas as pd

from collections import Counter
from datetime import datetime
from pathlib import Path

from gensim.models.doc2vec import Doc2Vec
# from hdbscan import HDBSCAN
from cuml.cluster import HDBSCAN, DBSCAN, KMeans
# from sklearn.decomposition import TruncatedSVD 
# from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import pairwise_distances, silhouette_score, calinski_harabasz_score, davies_bouldin_score
from cuml.metrics.cluster.silhouette_score import cython_silhouette_score
from cuml.metrics.cluster.entropy import cython_entropy

In [6]:
data_dir = Path(Path.cwd().parent, 'data/interim')
labels_dir = Path(data_dir, 'labels')
models_dir = Path(Path.cwd().parent, 'models')
mod_paths = sorted([mod_path for mod_path in Path(models_dir).glob('d2v*.model')])

In [18]:
df = pd.read_pickle(Path(data_dir, 'parsed_emails_chains_ge_4_lt_10.pkl'))

In [26]:
messages = df['Message'].to_numpy()

In [30]:
num_docs = len(messages)

In [31]:
mat = np.zeros((num_docs, num_docs))

In [32]:
mat.shape

(9676, 9676)

In [7]:
mod_paths

[PosixPath('/home/miguel/Projects/tfm-nlp/models/d2v_chains_all_300.model'),
 PosixPath('/home/miguel/Projects/tfm-nlp/models/d2v_chains_all_50.model'),
 PosixPath('/home/miguel/Projects/tfm-nlp/models/d2v_chains_eq_2_300.model'),
 PosixPath('/home/miguel/Projects/tfm-nlp/models/d2v_chains_eq_3_300.model'),
 PosixPath('/home/miguel/Projects/tfm-nlp/models/d2v_chains_ge_10_300.model'),
 PosixPath('/home/miguel/Projects/tfm-nlp/models/d2v_chains_ge_4_lt_10_300.model'),
 PosixPath('/home/miguel/Projects/tfm-nlp/models/d2v_chains_gt_1_300.model'),
 PosixPath('/home/miguel/Projects/tfm-nlp/models/d2v_chains_gt_1_50.model'),
 PosixPath('/home/miguel/Projects/tfm-nlp/models/d2v_chains_replies_300.model'),
 PosixPath('/home/miguel/Projects/tfm-nlp/models/d2v_chains_replies_50.model'),
 PosixPath('/home/miguel/Projects/tfm-nlp/models/d2v_chains_split_0_300.model'),
 PosixPath('/home/miguel/Projects/tfm-nlp/models/d2v_chains_split_0_50.model'),
 PosixPath('/home/miguel/Projects/tfm-nlp/models/d2

In [ ]:
docs = messages
m = []
for doc1 in docs:
    row = []
    for doc2 in docs:
        # if distance is similarity function
        row.append(model.dv.wmdistance(doc1, doc2))
    m.append(row)

result = np.matrix(m)

In [ ]:
mat = pairwise_distances(messages, metric=model.dv.wmdistance)

In [9]:
path = mod_paths[5]
path

PosixPath('/home/miguel/Projects/tfm-nlp/models/d2v_chains_ge_4_lt_10_300.model')

In [10]:
model = Doc2Vec.load(str(path))
data = model.dv.vectors

In [1]:
import csv
import pickle as pkl

import numpy as np
import pandas as pd
from sklearn.cluster import DBSCAN
from sklearn.metrics import pairwise_distances
from sklearn.metrics.pairwise import pairwise_kernels

from collections import Counter
from datetime import datetime
from pathlib import Path

from scipy.stats import wasserstein_distance
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.utils import simple_preprocess

In [6]:
data_dir = Path(Path.cwd().parent, 'data/interim')
labels_dir = Path(data_dir, 'labels')
models_dir = Path(Path.cwd().parent, 'models')
mod_paths = sorted([mod_path for mod_path in Path(models_dir).glob('d2v*.model')])[2:]
data_paths = sorted(data_dir.glob('parsed_emails*.pkl'))

In [8]:
data_paths[1]

PosixPath('/home/miguel/Projects/tfm-nlp/data/interim/parsed_emails_chains_gt_1.pkl')

In [3]:
mod_paths[1]


PosixPath('/home/miguel/Projects/tfm-nlp/models/d2v_chains_gt_1_50.model')

In [17]:
df_ = pd.read_pickle(data_paths[1])

In [18]:
df = df_[df_['Chain_len'] == 16]

In [20]:
df[~df['Reply']]

,Message,Reply,Chain,Chain_len,Subject,Sender,Recipients,Timestamp
90560,For your viewing pleasure . . .\n-------------...,False,126828,16,,eric.bass@enron.com,lwbthemarine@bigplanet.com,985038720
104474,"kate,\n\ni did not get to check out mike s. de...",False,218484,16,tfs,kerri.thompson@enron.com,kate.symes@enron.com,988045500
123035,hi matt....\n\n-cori\n(hopefully you remeber me),False,124316,16,,ccates@mail.lbjs.com,matthew.lenhart@enron.com,991155350
128118,Hi\n\nCan you can hit on Wed (lunch or afterno...,False,214079,16,meeting,elizabeth.sager@enron.com,drusso@reliantenergy.com,991781220


In [21]:

messages = df['Message'].dropna().to_numpy()
tokenized_docs = [simple_preprocess(msg) for msg in messages]
corpus = [TaggedDocument(doc, [i]) for i, doc in enumerate(tokenized_docs)]

In [25]:
vector_size = 50
window_size = 15
min_count = 1
train_epoch = 20
alpha = 0.25
min_alpha = 1e-5
model = Doc2Vec(vector_size=vector_size,
                window=window_size,
                alpha=alpha, 
                min_alpha=min_alpha,
                min_count=min_count,
                epochs=train_epoch,
                dm=0)
model.build_vocab(corpus)
model.train(corpus, total_examples=model.corpus_count, epochs=model.epochs)

In [26]:
# model = Doc2Vec.load(str(mod_paths[1]))
data = model.dv.vectors

In [27]:
data.shape

(112, 50)

In [28]:
db = DBSCAN(eps=0.3, min_samples=10, metric='cosine').fit(data)
labels = db.labels_

In [30]:
Counter(labels)

Counter({-1: 100, 0: 12})

In [31]:
dist_cosine = pairwise_distances(data, metric='cosine')

In [32]:
db = DBSCAN(eps=0.3, min_samples=10, metric='precomputed').fit(dist_cosine)
labels = db.labels_
Counter(labels)

Counter({-1: 100, 0: 12})

In [38]:
dist_wsd = np.asarray([[wasserstein_distance(p1, p2) for p2 in data] for p1 in data])

In [40]:
db = DBSCAN(eps=0.3, min_samples=10, metric='precomputed').fit(dist_wsd)
labels = db.labels_
Counter(labels)

Counter({0: 112})

In [ ]:
doclist1 = list(set(document1))
doclist2 = list(set(document2))
v1 = np.array([self.get_vector(token, norm=norm) for token in doclist1])
v2 = np.array([self.get_vector(token, norm=norm) for token in doclist2])
doc1_indices = dictionary.doc2idx(doclist1)
doc2_indices = dictionary.doc2idx(doclist2)

# Compute distance matrix.
distance_matrix = zeros((vocab_len, vocab_len), dtype=double)
distance_matrix[np.ix_(doc1_indices, doc2_indices)] = cdist(v1, v2)

if abs(np_sum(distance_matrix)) < 1e-8:
    # `emd` gets stuck if the distance matrix contains only zeros.
    logger.info('The distance matrix is all zeros. Aborting (returning inf).')
    return float('inf')

def nbow(document):
    d = zeros(vocab_len, dtype=double)
    nbow = dictionary.doc2bow(document)  # Word frequencies.
    doc_len = len(document)
    for idx, freq in nbow:
        d[idx] = freq / float(doc_len)  # Normalized word frequencies.
    return d